# Erklärung Main

Dieser Abschnitt erklärt die meisten Methoden des Objektes Main. Weggelassen wurden Methoden, welche nur für die Entwicklung benötigt wurden und im Produktivcode nicht aufgerufen werden (Debugging Methoden).
### Disclaimer
Der Code ist nicht lauffähig, dafür existieren die Python Skripte. Das Notebook stellt lediglich eine Dokumentation dar.

### Verbesserungen, welche an Main vorgenommen wurden
- Nichts besonderes

### Import aller spezifischen Bibliotheken und Module
Zu beachten sind calibration, sliding_window und hough_transformation

In [ ]:
import cv2 as cv
import os
import time

import calib as cal
import sliding_window as slw
import hough_transformation as hou

### Klassendeklaration und init Methode
Hier werden die Bildgrößen festgelegt und jeweils ein Objekt der oben importierten Module erzeugt. Der Pfad ist die URL für das Video, welches verwendet werden soll.

In [1]:
class Main():
    # Window size
    WIN_X = 1280
    WIN_Y = 720

    def __init__(self, path, debug=False):
        print('Willkommen beim Projekt "Erkennung von Spurmarkierungen"')
                
        # Define the objects
        self.calib = cal.Calibration(debug=debug)
        self.sliding_win = slw.SlidingWindow(debug=debug)
        self.hough = hou.HoughTransformation(debug = debug)
        
        # Define the variables
        self.path = path

### start video
Dies ist die Hauptfunktion des Objektes Main. 
Um die Spurerkennung zu starten, muss diese Methode aufgerufen werden. 
Grundsätzlich gibt es drei Modi: Hough, Sliding Windows und KI. 
Mittels des mode Parameter wird zwischen ihnen umgeschaltet. 
Je nach Modus werden die Konfigurationen des jeweiligen Ansatzes aus der Config Datei geladen. 
Eine Config-Datei existiert jeweils für ein Video. Darum stehen in einer Konfigurationsdatei alle Variablen für beide Ansätze. 
Danach wird das Video geladen und die Frames werden einzeln iteriert. 
Der erste Schritt für jedes Bild ist die Vorverarbeitung. 
Dabei wird der Frame in der Größe angepasst und dann Kalibriert (also entzerrt, nicht transformiert). 
Anschließend wird der Frame dann mit dem gewählten Ansatz bearbeitet. Zur detaillierten Beschreibung siehe Klassen Hough, SlidingWindows und KI.
Bei erfolgreicher Erkennung der Spur wird das Bild mit Fahrbahnmarkierung angezeigt, andernfalls ohne Markierung (aber vorverarbeitet). Es wird noch die berechnete FPS Zahl auf den Frame eingefügt und der Frame angezeigt. 

Der Export des Videos ist hier ebenfalls zu sehen. Es muss nur der Parameter export_video auf True gesetzt werden und das Video wird automatisch (ohne anzeigen der Frames für realistische Messung) unter jeweiligem Namen abgespeichert.

Wichtig zu erwähnen ist die Taste 'q' (Quit), welche die Wiedergabe des Videos abbricht.

In [ ]:
def startVideo(self, mode=0, config_path="./config/video.json", export_video=False):
        if not os.path.exists(self.path):
            return print('Video not found')
        
        # Load config
        error = None
        if mode==0:
            error = self.hough.load_config(config_path)
        elif mode == 1:
            error = self.sliding_win.load_config(config_path)
        elif mode == 2:
            pass
        else:
            error = "Mode not found"
        
        if error:
            print(error)
            return 

        # Load video
        video = cv.VideoCapture(self.path)
        prev_frame_time = 0
        new_frame_time = 0

        # Saving Video
        if export_video:
            saving_path = './Documentation/Videos/'
            mode_str = 'hough'
            vid_str = 'default_vid'
            if mode == 1:
                mode_str = 'sliding_windows'
            if mode == 2:
                mode_str = 'KI'
            if config_path == './config/video_challenge.json':
                vid_str = 'challenge_vid'
            filename = saving_path + vid_str + '_' + mode_str + '.avi'
            print(filename)
            out = cv.VideoWriter(filename, cv.VideoWriter_fourcc(*'MJPG'),25, (self.WIN_X, self.WIN_Y))

        # While the video is running
        while(video.isOpened()):
            ret, frame = video.read()

            # Break if video is finish or no input
            if not ret:
                break

            # Do here the image processing
            frame = cv.resize(frame, (self.WIN_X, self.WIN_Y))
            
            # Equalize the image
            frame = self.calib.equalize(frame)
            
            # Choose the mode
            if mode == 0:
                frame = self.hough.execute(frame)
            elif mode == 1:
                frame = self.sliding_win.execute(frame)
            elif mode == 2:
                frame = self.lane_detector.detectLanes(frame)
            else:
                return print('Mode not found')
            
            if (type(frame) == bool and not frame) or not frame.any():
                return print('Error: Module not loaded')

            # Do operations on the frame
            if frame is not None:
                font = cv.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()

                # Calculate Frame Rate
                fps = self._calcFPS(prev_frame_time, new_frame_time)
                

                # Put fps on the screen
                cv.putText(frame, fps, (7, 21), font, 1, (100, 100, 100), 2, cv.LINE_AA)
                
                if export_video:
                    frame = cv.resize(frame, (self.WIN_X, self.WIN_Y))
                    out.write(frame)
                else:
                    cv.imshow('Video', frame)
                    
                prev_frame_time = time.time()

            # press 'Q' for exit
            if cv.waitKey(1) & 0xFF == ord('q'):
                break


        # Stop video and close window
        video.release()
        if export_video:
            out.release()
        cv.destroyAllWindows()

### calc fps
In dieser Funktion werden die Frames per Second des Videos berechnet. Dafür wird von der Anzeigezeit des jetzigen Frames die Zeit des vorigen Frames subtrahiert.

In [ ]:
def _calcFPS(self, prev_frame_time, new_frame_time):
    # Calculate Frame Rate
    fps = 1/(new_frame_time-prev_frame_time)
    fps = int(fps)
    fps = str(fps)

    return fps

### Aufruf des main Objektes
Hier ist exemplarisch dargestellt, wie die Spurerkennung für ein Video gestartet werden kann. Es muss zuerst das Objekt erstellt werden, dabei muss auch der Dateipfad des Videos angegeben werden. Anschließend wird mit startVideo() die eigentliche Spurerkennung gestartet. Der Modus ist, ob als Ansatz Hough, Sliding Windows oder KI gewählt werden soll. 

In [ ]:
if __name__ == '__main__':
    # Path to video
    video = "img/Udacity/project_video.mp4"
    videoHarder = "img/Udacity/challenge_video.mp4"
    videoHardest = "img/Udacity/harder_challenge_video.mp4"
    
    # Start the program
    main = Main(video, debug=False)
    main2 = Main(videoHarder, debug=False)
    main3 = Main(videoHardest, debug=False)

    # Mode:
    # - 0: Hough
    # - 1: Sliding window
    # - 2: KI-Model
    main.startVideo(mode=0, config_path="./config/video.json")
    main.startVideo(mode=1, config_path="./config/video.json")
    main.startVideo(mode=2, config_path="./config/video.json")
    main2.startVideo(mode=0, config_path="./config/video_challenge.json")
    main2.startVideo(mode=1, config_path="./config/video_challenge.json")
    main2.startVideo(mode=2, config_path="./config/video_challenge.json")